## SPAM Filter

* Procedure
    * Divide data in train and test sets
    * Keep test data in a safe!
    * Transform test data (normalize, discretize, etc)
    * Train model
    * Transform test data with the parameters found in step 3
    * Test model with test data
    * Evaluate results

In [106]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from sklearn import preprocessing
import scipy as sp

In [114]:
df = pd.read_csv("spambase/spambase.data",header=None)

In [115]:
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[df.columns[-1]], train_size=0.75)

In [116]:
X_train=X_train.values
Y_train=Y_train.values
X_test=X_test.values
Y_test=Y_test.values


In [117]:
# Normalizar no ayuda mucho pero sale igual al de sklearn. Para que las alturas del pdf signifiquen lo mismo 
scaler = preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

Calcular las prior

In [118]:
P_Spam=float(sum(Y_train))/len(Y_train)
P_noSpam=1-P_Spam

Separar el Spam del No Spam

In [119]:
mediaS=np.mean(X_train[Y_train==1], axis=0)
dsS=np.std(X_train[Y_train==1],axis=0)
mediaNS=np.mean(X_train[Y_train==0],axis=0)
dsNS=np.std(X_train[Y_train==0],axis=0)

In [123]:
#para que no haya error el default muy negativo
def bayesClass(x):    
    min=-len(x)*10.0
    spam=np.log(P_Spam)+sum(np.log(norm(mediaS[i],dsS[i]).pdf(x[i]))  if norm(mediaS[i],dsS[i]).pdf(x[i]) <> 0.0  and dsS[i]>0.0  else min for i in range(len(x)))    
    noSpam=np.log(P_noSpam)+sum(np.log(norm(mediaNS[i],dsNS[i]).pdf(x[i])) if norm(mediaNS[i],dsNS[i]).pdf(x[i]) <> 0.0  and dsNS[i]>0.0 else min  for i in range(len(x)))
    if spam >= noSpam   :
            return 1
    else:
            return 0

In [124]:
#COn la integral. Asi no sirve bien. Pensar limites. No es necesario
#def bayesClass(x):    
#    spam=np.log(P_Spam)+sum(np.log(sp.integrate.quad(norm(mediaS[i],dsS[i]).pdf,x[i]-x[i]*0.001,x[i]+x[i]*0.001)[0])  if norm(mediaS[i],dsS[i]).pdf(x[i]) <> 0.0  and dsS[i]>0.0  else -10000.0 for i in range(len(x)))    
#    noSpam=np.log(P_noSpam)+sum(np.log(sp.integrate.quad(norm(mediaNS[i],dsNS[i]).pdf,x[i]-x[i]*0.001,x[i]+x[i]*0.001)[0]) if norm(mediaNS[i],dsNS[i]).pdf(x[i]) <> 0.0  and dsNS[i]>0.0 else -10000.0  for i in range(len(x)))
    
#    if spam > noSpam   :
#            return 1
#    else:
#            return 0

In [125]:
res=[bayesClass(x) for x in X_test]

In [126]:
confusion_matrix(Y_test, res)

array([[515, 170],
       [ 21, 445]])

In [127]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
modelo=gnb.fit(X_train,Y_train)

In [128]:
confusion_matrix(Y_test, modelo.predict(X_test))

array([[515, 170],
       [ 21, 445]])

In [113]:
x

0